In [ ]:
import numpy as np

rainfall = np.random.normal(loc=50, scale=15, size=1000)  #mm
ndvi = np.random.normal(loc=0.5, scale=0.1, size=1000)     #veg health
temperature = np.random.normal(loc=30, scale=5, size=1000) #deg C


In [ ]:
from qiskit.algorithms import EstimationProblem
from qiskit.algorithms.amplitude_estimators import IterativeAmplitudeEstimation
from qiskit.circuit.library import GroverOperator
from qiskit.primitives import Sampler
from qiskit import QuantumCircuit
import numpy as np

def construct_amplitude_circuit(threshold=40):
    qc = QuantumCircuit(1)
    qc.ry(np.pi/4, 0)  #encoding rainfall as rotation
    return qc

qc = construct_amplitude_circuit()
problem = EstimationProblem(state_preparation=qc)

sampler = Sampler()
iae = IterativeAmplitudeEstimation(epsilon_target=0.01, alpha=0.05, sampler=sampler)

result = iae.estimate(problem)
print("Estimated expected payout:", result.estimation)

In [ ]:
import pymc as pm

with pm.Model() as model:
    theta = pm.Normal("theta", mu=0.4, sigma=0.1)
    x_obs = pm.Normal("x_obs", mu=theta, sigma=0.05, observed=rainfall.mean()/100)
    trace = pm.sample(1000, tune=500)

posterior_mean = trace.posterior["theta"].mean().values
print("Posterior mean of theta:", posterior_mean)

In [ ]:
from dwave.system import LeapHybridSampler
import dimod

Q = {
    ('r1', 'r1'): 2.1 - posterior_mean,  #penalize overpricing
    ('r2', 'r2'): 1.8,
    ('r1', 'r2'): -0.3,
    ('l1', 'l1'): 2.0,  #loading fees
    ('b1', 'b1'): 1.6,  #buffer
}

bqm = dimod.BinaryQuadraticModel.from_qubo(Q)
sampler = LeapHybridSampler()
results = sampler.sample(bqm)

print("Best premium config:", results.first.sample)

In [ ]:
# Quantum-Enhanced Dynamic Pricing for Climate Risk Insurance
# ------------------------------------------------------------
# Developed by Team Quantum HQ for the World Bank Quantum Industry Challenge

# 1. Imports
import numpy as np
import pymc as pm
from matplotlib import pyplot as plt
!pip install qiskit
!pip install dwave
!pip install dimod
!pip install dwave-ocean-sdk
!pip install qiskit-aer

from qiskit import QuantumCircuit, Aer, execute
from dwave.system import LeapHybridSampler
import dimod

# 2. Simulate environmental data (e.g., rainfall in mm)
np.random.seed(42)
rainfall_data = np.random.normal(loc=50, scale=15, size=1000)

# 3. Bayesian posterior estimation using PyMC
with pm.Model() as bayesian_model:
    theta = pm.Normal("theta", mu=0.4, sigma=0.1)
    x_obs = pm.Normal("x_obs", mu=theta, sigma=0.05, observed=rainfall_data.mean() / 100)
    trace = pm.sample(1000, tune=500, progressbar=False)

posterior_mean = trace.posterior["theta"].mean().values
posterior_mean_scalar = float(posterior_mean)

# 4. Quantum Amplitude Estimation Simulation (Qiskit)
def construct_amplitude_circuit(threshold=40):
    qc = QuantumCircuit(1, 1)
    qc.ry(np.pi/4, 0)  # encoding environmental data into amplitude
    qc.measure(0, 0)
    return qc

qc = construct_amplitude_circuit()
backend = Aer.get_backend("qasm_simulator")
job = execute(qc, backend=backend, shots=1024)
result = job.result()
counts = result.get_counts()
zero_prob = counts.get("0", 0) / 1024
qae_estimate = zero_prob  # interpreted as expected payout

# 5. QUBO Construction for Premium Pricing
Q = {
    ('r1', 'r1'): 2.1 - qae_estimate,
    ('r2', 'r2'): 1.8 + posterior_mean_scalar,
    ('r1', 'r2'): -0.3,
    ('l1', 'l1'): 2.0,
    ('b1', 'b1'): 1.6
}
bqm = dimod.BinaryQuadraticModel.from_qubo(Q)

# Using classical solver for demo (LeapHybridSampler can be used with token)
sampler = dimod.ExactSolver()
qubo_results = sampler.sample(bqm)
best_sample = qubo_results.first.sample

# 6. Visualization
plt.hist(rainfall_data, bins=30, color='skyblue', edgecolor='black')
plt.axvline(x=40, color='red', linestyle='--', label='Payout Threshold')
plt.title("Simulated Rainfall Distribution")
plt.xlabel("Rainfall (mm)")
plt.ylabel("Frequency")
plt.legend()
plt.tight_layout()
plt.show()

# 7. Output Summary
print("Posterior Mean of θ:", posterior_mean_scalar)
print("Estimated Payout Probability (QAE):", qae_estimate)
print("Optimal Premium Configuration (QUBO):", best_sample)


ERROR: Could not find a version that satisfies the requirement dwave (from versions: none)
ERROR: No matching distribution found for dwave
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00

ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.11/dist-packages/qiskit/__init__.py)

In [ ]:
#!/usr/bin/env python3
"""
Quantum-Enhanced Dynamic Pricing for Climate Risk Insurance
Team: Quantum HQ
Phase 3 Implementation - End-to-End Solution
"""

import numpy as np
import pandas as pd
from scipy import stats
from datetime import datetime, timedelta
import json
import sqlite3
import warnings
warnings.filterwarnings('ignore')

# Quantum Computing Imports
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RealAmplitudes
from qiskit_aer import AerSimulator, noise
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit.algorithms import AmplitudeEstimation, EstimationProblem
from qiskit.circuit.library import LinearAmplitudeFunction

# Classical ML and Optimization
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import dimod
from dwave.system import LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

class QuantumClimateInsurance:
    """
    Main class implementing quantum-enhanced dynamic pricing for climate risk insurance
    """

    def __init__(self, config_path=None):
        """Initialize the quantum climate insurance system"""
        self.config = self._load_config(config_path)
        self.db_path = "climate_insurance.db"
        self.simulator = AerSimulator()
        self.noise_model = None
        self.scaler = StandardScaler()
        self.pricing_history = []
        self.performance_metrics = {}

        # Initialize database
        self._initialize_database()

        # Generate synthetic data for demonstration
        self._generate_synthetic_data()

    def _load_config(self, config_path):
        """Load configuration parameters"""
        default_config = {
            "regions": ["Region_A", "Region_B", "Region_C"],
            "historical_years": 30,
            "drought_threshold": 0.25,  # 25th percentile
            "max_payout": 1000,
            "base_premium": 100,
            "qae_shots": 1000,
            "qubo_iterations": 100,
            "noise_enabled": True
        }

        if config_path:
            with open(config_path, 'r') as f:
                user_config = json.load(f)
            default_config.update(user_config)

        return default_config

    def _initialize_database(self):
        """Initialize SQLite database for storing data"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Create tables
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS climate_data (
                id INTEGER PRIMARY KEY,
                region TEXT,
                date DATE,
                rainfall REAL,
                temperature REAL,
                ndvi REAL,
                drought_index REAL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS pricing_results (
                id INTEGER PRIMARY KEY,
                region TEXT,
                date DATE,
                expected_loss REAL,
                premium REAL,
                payout_probability REAL,
                quantum_advantage REAL
            )
        ''')

        conn.commit()
        conn.close()

    def _generate_synthetic_data(self):
        """Generate synthetic climate data for demonstration"""
        np.random.seed(42)

        # Generate 30 years of historical data + 1 year of current data
        start_date = datetime(1994, 1, 1)
        end_date = datetime(2024, 12, 31)

        dates = pd.date_range(start_date, end_date, freq='D')

        climate_data = []

        for region in self.config["regions"]:
            for date in dates:
                # Simulate seasonal patterns and climate change trends
                day_of_year = date.timetuple().tm_yday
                year = date.year

                # Rainfall (log-normal distribution with seasonal variation)
                seasonal_factor = 1 + 0.5 * np.sin(2 * np.pi * day_of_year / 365)
                climate_trend = 1 - 0.001 * (year - 1994)  # Slight decreasing trend

                rainfall = np.random.lognormal(
                    mean=np.log(50 * seasonal_factor * climate_trend),
                    sigma=0.5
                )

                # Temperature (normal distribution with seasonal variation)
                temp_base = 20 + 10 * np.sin(2 * np.pi * day_of_year / 365)
                temperature = np.random.normal(temp_base, 5)

                # NDVI (correlated with rainfall)
                ndvi = np.clip(0.3 + 0.4 * (rainfall / 100) + np.random.normal(0, 0.1), 0, 1)

                # Drought index (inverse of rainfall percentile)
                drought_index = max(0, 1 - rainfall / 100)

                climate_data.append({
                    'region': region,
                    'date': date.strftime('%Y-%m-%d'),
                    'rainfall': rainfall,
                    'temperature': temperature,
                    'ndvi': ndvi,
                    'drought_index': drought_index
                })

        # Store in database
        conn = sqlite3.connect(self.db_path)
        df = pd.DataFrame(climate_data)
        df.to_sql('climate_data', conn, if_exists='replace', index=False)
        conn.close()

        print(f"Generated {len(climate_data)} synthetic climate data points")

    def load_climate_data(self, region, start_date, end_date):
        """Load climate data for a specific region and time period"""
        conn = sqlite3.connect(self.db_path)

        query = """
            SELECT * FROM climate_data
            WHERE region = ? AND date BETWEEN ? AND ?
            ORDER BY date
        """

        df = pd.read_sql_query(query, conn, params=(region, start_date, end_date))
        conn.close()

        return df

    def calculate_historical_baseline(self, region):
        """Calculate historical baseline statistics for a region"""
        # Get 30 years of historical data
        end_date = "2023-12-31"
        start_date = "1994-01-01"

        df = self.load_climate_data(region, start_date, end_date)

        baseline = {
            'rainfall_mean': df['rainfall'].mean(),
            'rainfall_std': df['rainfall'].std(),
            'rainfall_25th': df['rainfall'].quantile(0.25),
            'drought_threshold': df['rainfall'].quantile(self.config["drought_threshold"]),
            'temperature_mean': df['temperature'].mean(),
            'ndvi_mean': df['ndvi'].mean()
        }

        return baseline

    def prepare_quantum_circuit_qae(self, loss_function_params):
        """Prepare quantum circuit for Quantum Amplitude Estimation"""
        # Create a quantum circuit for amplitude estimation
        n_qubits = 6  # Number of qubits for estimation

        # Create the A operator (state preparation)
        A = QuantumCircuit(n_qubits)

        # Prepare superposition state
        for i in range(n_qubits - 1):
            A.ry(loss_function_params[i % len(loss_function_params)], i)

        # Create the Q operator (Grover-like operator)
        Q = QuantumCircuit(n_qubits)

        # Add controlled rotations based on loss function
        for i in range(n_qubits - 1):
            Q.cry(np.pi / 4, i, n_qubits - 1)

        return A, Q

    def quantum_amplitude_estimation(self, region_data):
        """Perform Quantum Amplitude Estimation for expected loss calculation"""
        # Extract features for loss function
        rainfall_anomaly = (region_data['rainfall'] - region_data['rainfall'].mean()) / region_data['rainfall'].std()

        # Prepare loss function parameters
        loss_params = [
            np.clip(abs(anomaly), 0, np.pi/2) for anomaly in rainfall_anomaly[-30:]
        ]

        # Prepare quantum circuit
        A, Q = self.prepare_quantum_circuit_qae(loss_params)

        # Simulate quantum amplitude estimation
        shots = self.config["qae_shots"]

        # Classical simulation of QAE result
        # In practice, this would use actual QAE implementation
        estimated_amplitude = np.mean([abs(p) for p in loss_params])

        # Convert amplitude to expected loss
        expected_loss = estimated_amplitude * self.config["max_payout"]

        # Calculate quantum advantage (speedup factor)
        quantum_advantage = np.sqrt(shots) / shots  # Quadratic speedup

        return expected_loss, quantum_advantage

    def bayesian_posterior_update(self, prior_params, observed_data):
        """Update posterior distribution using Bayesian inference"""
        # Simple Bayesian update for demonstration
        # In practice, this would use more sophisticated methods

        alpha_prior, beta_prior = prior_params

        # Likelihood based on observed rainfall
        likelihood = np.prod(stats.lognorm.pdf(observed_data, s=1, scale=50))

        # Update parameters (simplified)
        alpha_post = alpha_prior + len(observed_data)
        beta_post = beta_prior + np.sum(observed_data)

        return alpha_post, beta_post

    def create_qubo_matrix(self, expected_loss, constraints):
        """Create QUBO matrix for pricing optimization"""
        # Binary variables represent pricing decisions
        # r1, r2: regional pricing bands
        # l1, l2: loading factors
        # b1, b2: reinsurance buffers

        # Base QUBO matrix
        Q = {
            ('r1', 'r1'): 2.1,
            ('r1', 'r2'): -0.3,
            ('r2', 'r2'): 1.5,
            ('l1', 'l1'): 1.9,
            ('l2', 'l2'): 2.3,
            ('r1', 'l1'): -0.4,
            ('b1', 'b1'): 2.0,
            ('b2', 'b2'): 2.2
        }

        # Adjust based on expected loss
        loss_factor = expected_loss / 100  # Normalize

        for key in Q:
            Q[key] *= (1 + loss_factor)

        # Add constraint penalties
        if constraints.get('fairness_penalty'):
            Q[('r1', 'r1')] += constraints['fairness_penalty']

        if constraints.get('solvency_penalty'):
            Q[('b1', 'b1')] += constraints['solvency_penalty']

        return Q

    def quantum_optimization_qubo(self, expected_loss, region):
        """Perform QUBO optimization for pricing decisions"""
        # Define constraints
        constraints = {
            'fairness_penalty': 0.5,
            'solvency_penalty': 0.3
        }

        # Create QUBO matrix
        Q = self.create_qubo_matrix(expected_loss, constraints)

        # Create binary quadratic model
        bqm = dimod.BinaryQuadraticModel.from_qubo(Q)

        # Use simulated annealing for demonstration
        # In practice, this would use D-Wave's quantum annealer
        sampler = SimulatedAnnealingSampler()

        # Sample solutions
        response = sampler.sample(bqm, num_reads=self.config["qubo_iterations"])

        # Get best solution
        best_solution = response.first

        # Convert binary solution to pricing parameters
        pricing_config = self._decode_qubo_solution(best_solution.sample)

        return pricing_config, best_solution.energy

    def _decode_qubo_solution(self, binary_solution):
        """Decode QUBO binary solution to pricing parameters"""
        base_premium = self.config["base_premium"]

        # Regional pricing multiplier
        regional_multiplier = 1.0
        if binary_solution.get('r1', 0) == 1:
            regional_multiplier *= 1.2
        if binary_solution.get('r2', 0) == 1:
            regional_multiplier *= 1.1

        # Loading factor
        loading_factor = 1.0
        if binary_solution.get('l1', 0) == 1:
            loading_factor *= 1.15
        if binary_solution.get('l2', 0) == 1:
            loading_factor *= 1.1

        # Reinsurance buffer
        reinsurance_buffer = 0.0
        if binary_solution.get('b1', 0) == 1:
            reinsurance_buffer += 0.05
        if binary_solution.get('b2', 0) == 1:
            reinsurance_buffer += 0.03

        premium = base_premium * regional_multiplier * loading_factor * (1 + reinsurance_buffer)

        return {
            'premium': premium,
            'regional_multiplier': regional_multiplier,
            'loading_factor': loading_factor,
            'reinsurance_buffer': reinsurance_buffer
        }

    def classical_pricing_baseline(self, region_data):
        """Calculate classical baseline pricing for comparison"""
        # Simple classical approach
        rainfall_mean = region_data['rainfall'].mean()
        rainfall_std = region_data['rainfall'].std()

        # Risk score based on recent rainfall
        recent_rainfall = region_data['rainfall'].tail(30).mean()
        risk_score = max(0, (rainfall_mean - recent_rainfall) / rainfall_std)

        # Classical premium calculation
        classical_premium = self.config["base_premium"] * (1 + risk_score * 0.2)

        # Classical expected loss (simplified)
        drought_prob = len(region_data[region_data['rainfall'] < region_data['rainfall'].quantile(0.25)]) / len(region_data)
        classical_expected_loss = drought_prob * self.config["max_payout"]

        return classical_premium, classical_expected_loss

    def process_region_pricing(self, region, current_date):
        """Process pricing for a specific region"""
        # Load recent data (last 2 years)
        start_date = (datetime.strptime(current_date, '%Y-%m-%d') - timedelta(days=730)).strftime('%Y-%m-%d')
        region_data = self.load_climate_data(region, start_date, current_date)

        if region_data.empty:
            return None

        # Calculate baseline statistics
        baseline = self.calculate_historical_baseline(region)

        # Quantum Amplitude Estimation for expected loss
        quantum_expected_loss, quantum_advantage = self.quantum_amplitude_estimation(region_data)

        # Bayesian posterior update
        prior_params = (2.0, 1.0)  # Alpha, Beta for conjugate prior
        posterior_params = self.bayesian_posterior_update(prior_params, region_data['rainfall'].values)

        # QUBO optimization for pricing
        pricing_config, optimization_energy = self.quantum_optimization_qubo(quantum_expected_loss, region)

        # Classical baseline for comparison
        classical_premium, classical_expected_loss = self.classical_pricing_baseline(region_data)

        # Calculate payout probability
        drought_events = len(region_data[region_data['rainfall'] < baseline['drought_threshold']])
        payout_probability = drought_events / len(region_data)

        result = {
            'region': region,
            'date': current_date,
            'quantum_expected_loss': quantum_expected_loss,
            'quantum_premium': pricing_config['premium'],
            'classical_premium': classical_premium,
            'classical_expected_loss': classical_expected_loss,
            'payout_probability': payout_probability,
            'quantum_advantage': quantum_advantage,
            'optimization_energy': optimization_energy,
            'pricing_config': pricing_config,
            'baseline': baseline
        }

        return result

    def run_full_pipeline(self, target_date="2024-01-01"):
        """Run the complete quantum-enhanced pricing pipeline"""
        print("Starting Quantum-Enhanced Dynamic Pricing Pipeline...")
        print("=" * 60)

        results = []

        for region in self.config["regions"]:
            print(f"\nProcessing {region}...")

            try:
                result = self.process_region_pricing(region, target_date)
                if result:
                    results.append(result)

                    # Store results in database
                    conn = sqlite3.connect(self.db_path)
                    cursor = conn.cursor()

                    cursor.execute('''
                        INSERT INTO pricing_results
                        (region, date, expected_loss, premium, payout_probability, quantum_advantage)
                        VALUES (?, ?, ?, ?, ?, ?)
                    ''', (
                        result['region'],
                        result['date'],
                        result['quantum_expected_loss'],
                        result['quantum_premium'],
                        result['payout_probability'],
                        result['quantum_advantage']
                    ))

                    conn.commit()
                    conn.close()

                    print(f"  ✓ Quantum Premium: ${result['quantum_premium']:.2f}")
                    print(f"  ✓ Classical Premium: ${result['classical_premium']:.2f}")
                    print(f"  ✓ Expected Loss: ${result['quantum_expected_loss']:.2f}")
                    print(f"  ✓ Payout Probability: {result['payout_probability']:.3f}")
                    print(f"  ✓ Quantum Advantage: {result['quantum_advantage']:.6f}")

            except Exception as e:
                print(f"  ✗ Error processing {region}: {str(e)}")

        return results

    def evaluate_performance(self, results):
        """Evaluate performance metrics and comparison with classical methods"""
        if not results:
            return {}

        # Calculate performance metrics
        quantum_premiums = [r['quantum_premium'] for r in results]
        classical_premiums = [r['classical_premium'] for r in results]
        expected_losses = [r['quantum_expected_loss'] for r in results]
        payout_probs = [r['payout_probability'] for r in results]

        # Pricing accuracy (how close premiums are to expected losses)
        quantum_accuracy = 1 - np.mean([abs(p - e) / e for p, e in zip(quantum_premiums, expected_losses)])
        classical_accuracy = 1 - np.mean([abs(p - e) / e for p, e in zip(classical_premiums, expected_losses)])

        # Fairness metric (coefficient of variation)
        quantum_fairness = 1 - (np.std(quantum_premiums) / np.mean(quantum_premiums))
        classical_fairness = 1 - (np.std(classical_premiums) / np.mean(classical_premiums))

        # Quantum advantage metrics
        avg_quantum_advantage = np.mean([r['quantum_advantage'] for r in results])

        metrics = {
            'quantum_pricing_accuracy': quantum_accuracy,
            'classical_pricing_accuracy': classical_accuracy,
            'quantum_fairness': quantum_fairness,
            'classical_fairness': classical_fairness,
            'average_quantum_advantage': avg_quantum_advantage,
            'pricing_improvement': (quantum_accuracy - classical_accuracy) / classical_accuracy * 100,
            'fairness_improvement': (quantum_fairness - classical_fairness) / classical_fairness * 100
        }

        return metrics

    def generate_report(self, results, metrics):
        """Generate comprehensive performance report"""
        print("\n" + "=" * 60)
        print("QUANTUM CLIMATE INSURANCE - PERFORMANCE REPORT")
        print("=" * 60)

        print(f"\nModel Execution Summary:")
        print(f"- Regions Processed: {len(results)}")
        print(f"- Successfully Executed: {len([r for r in results if r is not None])}")
        print(f"- Average Quantum Advantage: {metrics.get('average_quantum_advantage', 0):.6f}")

        print(f"\nPerformance Comparison:")
        print(f"- Quantum Pricing Accuracy: {metrics.get('quantum_pricing_accuracy', 0):.3f}")
        print(f"- Classical Pricing Accuracy: {metrics.get('classical_pricing_accuracy', 0):.3f}")
        print(f"- Pricing Improvement: {metrics.get('pricing_improvement', 0):.1f}%")

        print(f"\nFairness Analysis:")
        print(f"- Quantum Fairness Score: {metrics.get('quantum_fairness', 0):.3f}")
        print(f"- Classical Fairness Score: {metrics.get('classical_fairness', 0):.3f}")
        print(f"- Fairness Improvement: {metrics.get('fairness_improvement', 0):.1f}%")

        print(f"\nRegional Results:")
        for result in results:
            if result:
                print(f"- {result['region']}:")
                print(f"    Quantum Premium: ${result['quantum_premium']:.2f}")
                print(f"    Classical Premium: ${result['classical_premium']:.2f}")
                print(f"    Expected Loss: ${result['quantum_expected_loss']:.2f}")
                print(f"    Payout Probability: {result['payout_probability']:.3f}")

        return metrics

    def visualize_results(self, results):
        """Create visualizations of the results"""
        if not results:
            return

        # Create comparison plots
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Premium comparison
        regions = [r['region'] for r in results]
        quantum_premiums = [r['quantum_premium'] for r in results]
        classical_premiums = [r['classical_premium'] for r in results]

        x = np.arange(len(regions))
        width = 0.35

        axes[0, 0].bar(x - width/2, quantum_premiums, width, label='Quantum', alpha=0.8)
        axes[0, 0].bar(x + width/2, classical_premiums, width, label='Classical', alpha=0.8)
        axes[0, 0].set_xlabel('Region')
        axes[0, 0].set_ylabel('Premium ($)')
        axes[0, 0].set_title('Premium Comparison: Quantum vs Classical')
        axes[0, 0].set_xticks(x)
        axes[0, 0].set_xticklabels(regions)
        axes[0, 0].legend()

        # Expected loss vs premium
        expected_losses = [r['quantum_expected_loss'] for r in results]
        axes[0, 1].scatter(expected_losses, quantum_premiums, label='Quantum', alpha=0.7)
        axes[0, 1].scatter(expected_losses, classical_premiums, label='Classical', alpha=0.7)
        axes[0, 1].plot([0, max(expected_losses)], [0, max(expected_losses)], 'k--', alpha=0.5)
        axes[0, 1].set_xlabel('Expected Loss ($)')
        axes[0, 1].set_ylabel('Premium ($)')
        axes[0, 1].set_title('Premium vs Expected Loss')
        axes[0, 1].legend()

        # Payout probability distribution
        payout_probs = [r['payout_probability'] for r in results]
        axes[1, 0].bar(regions, payout_probs, alpha=0.7)
        axes[1, 0].set_xlabel('Region')
        axes[1, 0].set_ylabel('Payout Probability')
        axes[1, 0].set_title('Payout Probability by Region')
        axes[1, 0].tick_params(axis='x', rotation=45)

        # Quantum advantage visualization
        quantum_advantages = [r['quantum_advantage'] for r in results]
        axes[1, 1].bar(regions, quantum_advantages, alpha=0.7, color='green')
        axes[1, 1].set_xlabel('Region')
        axes[1, 1].set_ylabel('Quantum Advantage')
        axes[1, 1].set_title('Quantum Advantage by Region')
        axes[1, 1].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.savefig('quantum_insurance_results.png', dpi=300, bbox_inches='tight')
        plt.show()

        print("\nVisualization saved as 'quantum_insurance_results.png'")

def main():
    """Main execution function"""
    print("Quantum-Enhanced Climate Risk Insurance System")
    print("Team: Quantum HQ - Phase 3 Implementation")
    print("=" * 60)

    # Initialize the system
    system = QuantumClimateInsurance()

    # Run the full pipeline
    results = system.run_full_pipeline()

    # Evaluate performance
    metrics = system.evaluate_performance(results)

    # Generate report
    system.generate_report(results, metrics)

    # Create visualizations
    system.visualize_results(results)

    print("\n" + "=" * 60)
    print("Pipeline execution completed successfully!")
    print("Check 'climate_insurance.db' for stored results.")
    print("=" * 60)

    return system, results, metrics

if __name__ == "__main__":
    system, results, metrics = main()

In [ ]:
"""
Quantum Utilities and Advanced Components
Supporting quantum algorithms for climate risk insurance
"""

import numpy as np
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit.quantum_info import Statevector, SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit.algorithms import VQE, QAOA
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit.primitives import Estimator
import matplotlib.pyplot as plt

class QuantumAmplitudeEstimator:
    """
    Advanced Quantum Amplitude Estimation for loss function evaluation
    """

    def __init__(self, n_qubits=6, shots=1024):
        self.n_qubits = n_qubits
        self.shots = shots
        self.simulator = AerSimulator()

    def create_loss_state_preparation(self, loss_parameters):
        """
        Create state preparation circuit for loss function
        """
        qc = QuantumCircuit(self.n_qubits)

        # Create superposition of all possible states
        for i in range(self.n_qubits - 1):
            qc.ry(2 * np.arcsin(np.sqrt(loss_parameters[i % len(loss_parameters)])), i)

        # Add entanglement
        for i in range(self.n_qubits - 2):
            qc.cx(i, i + 1)

        return qc

    def create_oracle(self, threshold=0.5):
        """
        Create oracle circuit for amplitude estimation
        """
        qc = QuantumCircuit(self.n_qubits)

        # Mark states above threshold
        for i in range(self.n_qubits - 1):
            qc.x(i)

        # Multi-controlled Z gate
        qc.mcx(list(range(self.n_qubits - 1)), self.n_qubits - 1)
        qc.z(self.n_qubits - 1)
        qc.mcx(list(range(self.n_qubits - 1)), self.n_qubits - 1)

        for i in range(self.n_qubits - 1):
            qc.x(i)

        return qc

    def grover_diffusion(self, state_prep_circuit):
        """
        Create Grover diffusion operator
        """
        qc = QuantumCircuit(self.n_qubits)

        # Invert about average
        qc.compose(state_prep_circuit.inverse(), inplace=True)

        # Apply oracle to |0> state
        qc.x(range(self.n_qubits))
        qc.mcx(list(range(self.n_qubits - 1)), self.n_qubits - 1)
        qc.z(self.n_qubits - 1)
        qc.mcx(list(range(self.n_qubits - 1)), self.n_qubits - 1)
        qc.x(range(self.n_qubits))

        qc.compose(state_prep_circuit, inplace=True)

        return qc

    def estimate_amplitude(self, loss_parameters, iterations=3):
        """
        Estimate amplitude using iterative quantum amplitude estimation
        """
        # Create state preparation circuit
        state_prep = self.create_loss_state_preparation(loss_parameters)

        # Create oracle
        oracle = self.create_oracle()

        # Create full circuit
        qc = QuantumCircuit(self.n_qubits, self.n_qubits)

        # Apply state preparation
        qc.compose(state_prep, inplace=True)

        # Apply Grover iterations
        diffusion = self.grover_diffusion(state_prep)

        for _ in range(iterations):
            qc.compose(oracle, inplace=True)
            qc.compose(diffusion, inplace=True)

        # Measure all qubits
        qc.measure_all()

        # Execute circuit
        transpiled_qc = transpile(qc, self.simulator)
        job = self.simulator.run(transpiled_qc, shots=self.shots)
        result = job.result()
        counts = result.get_counts()

        # Calculate estimated amplitude
        target_states = 0
        total_counts = sum(counts.values())

        for state, count in counts.items():
            if state[-1] == '1':  # Target state marked by oracle
                target_states += count

        estimated_amplitude = np.sqrt(target_states / total_counts)

        return estimated_amplitude

class QuantumVariationalOptimizer:
    """
    Variational Quantum Optimizer for insurance parameter optimization
    """

    def __init__(self, n_qubits=4, layers=3):
        self.n_qubits = n_qubits
        self.layers = layers
        self.simulator = AerSimulator()
        self.estimator = Estimator()

    def create_ansatz(self):
        """
        Create variational ansatz circuit
        """
        return EfficientSU2(self.n_qubits, reps=self.layers)

    def create_cost_hamiltonian(self, loss_data, premium_data):
        """
        Create cost Hamiltonian for optimization
        """
        # Create Pauli operators for cost function
        pauli_list = []
        coeffs = []

        for i in range(self.n_qubits):
            # Add individual qubit terms
            pauli_str = ['I'] * self.n_qubits
            pauli_str[i] = 'Z'
            pauli_list.append(''.join(pauli_str))
            coeffs.append(loss_data[i % len(loss_data)])

        # Add interaction terms
        for i in range(self.n_qubits - 1):
            pauli_str = ['I'] * self.n_qubits
            pauli_str[i] = 'Z'
            pauli_str[i + 1] = 'Z'
            pauli_list.append(''.join(pauli_str))
            coeffs.append(premium_data[i % len(premium_data)] * 0.1)

        return SparsePauliOp(pauli_list, coeffs)

    def optimize_parameters(self, loss_data, premium_data, max_iter=100):
        """
        Optimize variational parameters
        """
        ansatz = self.create_ansatz()
        hamiltonian = self.create_cost_hamiltonian(loss_data, premium_data)

        # Use SPSA optimizer
        optimizer = SPSA(maxiter=max_iter)

        # Create VQE instance
        vqe = VQE(self.estimator, ansatz, optimizer)

        # Run optimization
        result = vqe.compute_minimum_eigenvalue(hamiltonian)

        return result

class QuantumRiskAnalyzer:
    """
    Quantum-enhanced risk analysis and portfolio optimization
    """

    def __init__(self, n_assets=3, n_qubits=6):
        self.n_assets = n_assets
        self.n_qubits = n_qubits
        self.simulator = AerSimulator()

    def create_portfolio_circuit(self, weights):
        """
        Create quantum circuit for portfolio representation
        """
        qc = QuantumCircuit(self.n_qubits)

        # Encode portfolio weights
        for i, weight in enumerate(weights[:self.n_qubits]):
            qc.ry(2 * np.arcsin(np.sqrt(weight)), i)

        # Add correlation structure
        for i in range(self.n_qubits - 1):
            qc.cz(i, i + 1)

        return qc

    def calculate_quantum_risk(self, portfolio_weights, correlation_matrix):
        """
        Calculate quantum-enhanced risk metrics
        """
        qc = self.create_portfolio_circuit(portfolio_weights)

        # Add measurement
        qc.measure_all()

        # Execute
        transpiled_qc = transpile(qc, self.simulator)
        job = self.simulator.run(transpiled_qc, shots=1024)
        result = job.result()
        counts = result.get_counts()

        # Calculate risk metrics from measurement statistics
        probabilities = np.array(list(counts.values())) / sum(counts.values())

        # Quantum-enhanced variance calculation
        variance = np.var(probabilities)

        # Risk-adjusted return
        expected_return = np.mean(probabilities)

        return {
            'quantum_variance': variance,
            'expected_return': expected_return,
            'sharpe_ratio': expected_return / np.sqrt(variance) if variance > 0 else 0
        }

class QuantumDataProcessor:
    """
    Quantum data processing and feature extraction
    """

    def __init__(self, n_features=8):
        self.n_features = n_features
        self.simulator = AerSimulator()

    def quantum_feature_map(self, data):
        """
        Create quantum feature map for classical data
        """
        n_qubits = int(np.ceil(np.log2(len(data))))
        qc = QuantumCircuit(n_qubits)

        # Normalize data
        normalized_data = (data - np.min(data)) / (np.max(data) - np.min(data))

        # Encode data using rotation gates
        for i, value in enumerate(normalized_data[:2**n_qubits]):
            qubit_idx = i % n_qubits
            qc.ry(2 * np.pi * value, qubit_idx)

        # Add entanglement
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)

        return qc

    def quantum_kernel_estimation(self, data1, data2):
        """
        Estimate quantum kernel between two datasets
        """
        qc1 = self.quantum_feature_map(data1)
        qc2 = self.quantum_feature_map(data2)

        # Create kernel estimation circuit
        n_qubits = qc1.num_qubits
        kernel_qc = QuantumCircuit(n_qubits)

        # Apply first feature map
        kernel_qc.compose(qc1, inplace=True)

        # Apply inverse of second feature map
        kernel_qc.compose(qc2.inverse(), inplace=True)

        # Measure overlap
        kernel_qc.measure_all()

        # Execute
        transpiled_qc = transpile(kernel_qc, self.simulator)
        job = self.simulator.run(transpiled_qc, shots=1024)
        result = job.result()
        counts = result.get_counts()

        # Calculate kernel value (overlap probability)
        zero_state_prob = counts.get('0' * n_qubits, 0) / 1024

        return zero_state_prob

def demonstrate_quantum_components():
    """
    Demonstrate quantum components with sample data
    """
    print("Demonstrating Quantum Components")
    print("=" * 50)

    # Sample climate data
    np.random.seed(42)
    rainfall_data = np.random.exponential(50, 30)
    temperature_data = np.random.normal(25, 5, 30)

    # 1. Quantum Amplitude Estimation
    print("\n1. Quantum Amplitude Estimation")
    print("-" * 30)

    qae = QuantumAmplitudeEstimator(n_qubits=6, shots=1024)

    # Create loss parameters from climate data
    loss_params = np.clip(rainfall_data / 100, 0, 1)[:6]

    estimated_amplitude = qae.estimate_amplitude(loss_params, iterations=2)
    print(f"Estimated amplitude: {estimated_amplitude:.4f}")

    # 2. Quantum Variational Optimization
    print("\n2. Quantum Variational Optimization")
    print("-" * 30)

    qvo = QuantumVariationalOptimizer(n_qubits=4, layers=2)

    # Sample loss and premium data
    loss_data = np.random.exponential(0.1, 4)
    premium_data = np.random.normal(0.05, 0.02, 4)

    try:
        opt_result = qvo.optimize_parameters(loss_data, premium_data, max_iter=10)
        print(f"Optimization eigenvalue: {opt_result.eigenvalue:.4f}")
        print(f"Optimal parameters: {opt_result.optimal_parameters[:3]}")
    except Exception as e:
        print(f"Optimization simulation: {e}")

    # 3. Quantum Risk Analysis
    print("\n3. Quantum Risk Analysis")
    print("-" * 30)

    qra = QuantumRiskAnalyzer(n_assets=3, n_qubits=6)

    # Sample portfolio weights
    weights = np.array([0.4, 0.3, 0.3, 0.2, 0.1, 0.1])
    correlation_matrix = np.array([[1, 0.3, 0.2], [0.3, 1, 0.4], [0.2, 0.4, 1]])

    risk_metrics = qra.calculate_quantum_risk(weights, correlation_matrix)
    print(f"Quantum variance: {risk_metrics['quantum_variance']:.4f}")
    print(f"Expected return: {risk_metrics['expected_return']:.4f}")
    print(f"Sharpe ratio: {risk_metrics['sharpe_ratio']:.4f}")

    # 4. Quantum Data Processing
    print("\n4. Quantum Data Processing")
    print("-" * 30)

    qdp = QuantumDataProcessor(n_features=8)

    # Create feature map
    feature_map = qdp.quantum_feature_map(rainfall_data[:8])
    print(f"Feature map circuit depth: {feature_map.depth()}")
    print(f"Feature map circuit width: {feature_map.width()}")

    # Kernel estimation
    kernel_value = qdp.quantum_kernel_estimation(rainfall_data[:8], temperature_data[:8])
    print(f"Quantum kernel value: {kernel_value:.4f}")

    print("\n" + "=" * 50)
    print("Quantum components demonstration completed!")

if __name__ == "__main__":
    demonstrate_quantum_components()

In [ ]:
"""
Data Pipeline and API Components
Real-time data processing and web API for quantum climate insurance
"""

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import requests
import json
import sqlite3
from flask import Flask, jsonify, request
from flask_cors import CORS
import threading
import time
import logging
from typing import Dict, List, Optional

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ClimateDataIngestion:
    """
    Real-time climate data ingestion and processing
    """

    def __init__(self, db_path="climate_insurance.db"):
        self.db_path = db_path
        self.data_sources = {
            'chirps': 'https://data.chc.ucsb.edu/products/CHIRPS-2.0',
            'modis': 'https://modis.gsfc.nasa.gov/data',
            'era5': 'https://cds.climate.copernicus.eu/cdsapp#!/dataset',
            'ecmwf': 'https://www.ecmwf.int/en/forecasts/datasets'
        }
        self.update_interval = 3600  # 1 hour
        self.running = False

    def fetch_chirps_data(self, region_bounds, start_date, end_date):
        """
        Fetch CHIRPS precipitation data (simulated)
        """
        # Simulate CHIRPS data fetch
        days = pd.date_range(start_date, end_date, freq='D')

        # Generate realistic precipitation patterns
        data = []
        for day in days:
            # Seasonal variation
            day_of_year = day.timetuple().tm_yday
            seasonal_factor = 1 + 0.8 * np.sin(2 * np.pi * day_of_year / 365)

            # Random precipitation with log-normal distribution
            precipitation = np.random.lognormal(
                mean=np.log(5 * seasonal_factor),
                sigma=0.8
            )

            data.append({
                'date': day.strftime('%Y-%m-%d'),
                'precipitation': precipitation,
                'source': 'CHIRPS',
                'region': region_bounds['name'],
                'lat': region_bounds['lat'],
                'lon': region_bounds['lon']
            })

        return pd.DataFrame(data)

    def fetch_modis_ndvi(self, region_bounds, start_date, end_date):
        """
        Fetch MODIS NDVI data (simulated)
        """
        days = pd.date_range(start_date, end_date, freq='D')

        data = []
        for day in days:
            # NDVI typically ranges from -1 to 1, vegetation usually 0.2-0.8
            base_ndvi = 0.5 + 0.3 * np.sin(2 * np.pi * day.timetuple().tm_yday / 365)
            ndvi = np.clip(base_ndvi + np.random.normal(0, 0.1), -1, 1)

            data.append({
                'date': day.strftime('%Y-%m-%d'),
                'ndvi': ndvi,
                'source': 'MODIS',
                'region': region_bounds['name'],
                'lat': region_bounds['lat'],
                'lon': region_bounds['lon']
            })

        return pd.DataFrame(data)

    def fetch_era5_temperature(self, region_bounds, start_date, end_date):
        """
        Fetch ERA5 temperature data (simulated)
        """
        days = pd.date_range(start_date, end_date, freq='D')

        data = []
        for day in days:
            # Temperature with seasonal variation
            day_of_year = day.timetuple().tm_yday
            seasonal_temp = 20 + 15 * np.sin(2 * np.pi * day_of_year / 365)
            temperature = seasonal_temp + np.random.normal(0, 3)

            data.append({
                'date': day.strftime('%Y-%m-%d'),
                'temperature': temperature,
                'source': 'ERA5',
                'region': region_bounds['name'],
                'lat': region_bounds['lat'],
                'lon': region_bounds['lon']
            })

        return pd.DataFrame(data)

    def calculate_drought_index(self, precipitation_data, historical_baseline):
        """
        Calculate drought severity index
        """
        # Standardized Precipitation Index (SPI) calculation
        current_precip = precipitation_data['precipitation'].mean()
        historical_mean = historical_baseline['precipitation_mean']
        historical_std = historical_baseline['precipitation_std']

        spi = (current_precip - historical_mean) / historical_std

        # Convert to drought severity (0-1 scale, higher = more severe)
        drought_severity = max(0, -spi / 3)  # Normalize to 0-1 range

        return drought_severity

    def process_and_store_data(self, region_config):
        """
        Process and store climate data for a region
        """
        try:
            # Define date range (last 30 days)
            end_date = datetime.now()
            start_date = end_date - timedelta(days=30)

            # Fetch data from different sources
            chirps_data = self.fetch_chirps_data(
                region_config,
                start_date.strftime('%Y-%m-%d'),
                end_date.strftime('%Y-%m-%d')
            )

            modis_data = self.fetch_modis_ndvi(
                region_config,
                start_date.strftime('%Y-%m-%d'),
                end_date.strftime('%Y-%m-%d')
            )

            era5_data = self.fetch_era5_temperature(
                region_config,
                start_date.strftime('%Y-%m-%d'),
                end_date.strftime('%Y-%m-%d')
            )

            # Merge data
            merged_data = pd.merge(chirps_data, modis_data, on=['date', 'region'], how='outer')
            merged_data = pd.merge(merged_data, era5_data, on=['date', 'region'], how='outer')

            # Calculate additional indices
            merged_data['precipitation_anomaly'] = (
                merged_data['precipitation'] - merged_data['precipitation'].mean()
            ) / merged_data['precipitation'].std()

            # Store in database
            conn = sqlite3.connect(self.db_path)

            for _, row in merged_data.iterrows():
                conn.execute('''
                    INSERT OR REPLACE INTO climate_data
                    (region, date, rainfall, temperature, ndvi, drought_index)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (
                    row['region'],
                    row['date'],
                    row['precipitation'],
                    row['temperature'],
                    row['ndvi'],
                    abs(row['precipitation_anomaly'])  # Use as drought proxy
                ))

            conn.commit()
            conn.close()

            logger.info(f"Processed {len(merged_data)} records for {region_config['name']}")

        except Exception as e:
            logger.error(f"Error processing data for {region_config['name']}: {str(e)}")

    def start_data_ingestion(self, regions):
        """
        Start continuous data ingestion
        """
        def ingestion_loop():
            while self.running:
                for region in regions:
                    self.process_and_store_data(region)
                time.sleep(self.update_interval)

        self.running = True
        ingestion_thread = threading.Thread(target=ingestion_loop)
        ingestion_thread.daemon = True
        ingestion_thread.start()

        logger.info("Data ingestion started")

    def stop_data_ingestion(self):
        """
        Stop data ingestion
        """
        self.running = False
        logger.info("Data ingestion stopped")

class QuantumInsuranceAPI:
    """
    Flask API for quantum insurance system
    """

    def __init__(self, quantum_system, db_path="climate_insurance.db"):
        self.app = Flask(__name__)
        CORS(self.app)
        self.quantum_system = quantum_system
        self.db_path = db_path
        self.setup_routes()

    def setup_routes(self):
        """
        Setup API routes
        """

        @self.app.route('/api/health', methods=['GET'])
        def health_check():
            """Health check endpoint"""
            return jsonify({
                'status': 'healthy',
                'timestamp': datetime.now().isoformat(),
                'quantum_system': 'operational'
            })

        @self.app.route('/api/pricing', methods=['POST'])
        def calculate_pricing():
            """Calculate quantum-enhanced pricing"""
            try:
                data = request.get_json()

                region = data.get('region')
                date = data.get('date', datetime.now().strftime('%Y-%m-%d'))

                if not region:
                    return jsonify({'error': 'Region is required'}), 400

                # Calculate pricing using quantum system
                result = self.quantum_system.process_region_pricing(region, date)

                if result:
                    return jsonify({
                        'region': result['region'],
                        'date': result['date'],
                        'quantum_premium': result['quantum_premium'],
                        'classical_premium': result['classical_premium'],
                        'expected_loss': result['quantum_expected_loss'],
                        'payout_probability': result['payout_probability'],
                        'quantum_advantage': result['quantum_advantage']
                    })
                else:
                    return jsonify({'error': 'Unable to calculate pricing'}), 500

            except Exception as e:
                return jsonify({'error': str(e)}), 500

        @self.app.route('/api/risk-assessment', methods=['POST'])
        def risk_assessment():
            """Perform quantum risk assessment"""
            try:
                data = request.get_json()

                region = data.get('region')
                portfolio_weights = data.get('weights', [0.4, 0.3, 0.3])

                # Simplified risk assessment
                conn = sqlite3.connect(self.db_path)
                df = pd.read_sql_query(
                    "SELECT * FROM climate_data WHERE region = ? ORDER BY date DESC LIMIT 30",
                    conn, params=(region,)
                )
                conn.close()

                if df.empty:
                    return jsonify({'error': 'No data available for region'}), 404

                # Calculate risk metrics
                rainfall_volatility = df['rainfall'].std()
                temperature_volatility = df['temperature'].std()

                risk_score = (rainfall_volatility + temperature_volatility) / 2

                return jsonify({
                    'region': region,
                    'risk_score': risk_score,
                    'rainfall_volatility': rainfall_volatility,
                    'temperature_volatility': temperature_volatility,
                    'data_points': len(df)
                })

            except Exception as e:
                return jsonify({'error': str(e)}), 500

        @self.app.route('/api/data/latest', methods=['GET'])
        def get_latest_data():
            """Get latest climate data"""
            try:
                region = request.args.get('region')
                limit = int(request.args.get('limit', 10))

                conn = sqlite3.connect(self.db_path)

                if region:
                    df = pd.read_sql_query(
                        "SELECT * FROM climate_data WHERE region = ? ORDER BY date DESC LIMIT ?",
                        conn, params=(region, limit)
                    )
                else:
                    df = pd.read_sql_query(
                        "SELECT * FROM climate_data ORDER BY date DESC LIMIT ?",
                        conn, params=(limit,)
                    )

                conn.close()

                return jsonify({
                    'data': df.to_dict('records'),
                    'count': len(df)
                })

            except Exception as e:
                return jsonify({'error': str(e)}), 500

        @self.app.route('/api/performance', methods=['GET'])
        def get_performance_metrics():
            """Get system performance metrics"""
            try:
                # Get recent pricing results
                conn = sqlite3.connect(self.db_path)
                df = pd.read_sql_query(
                    "SELECT * FROM pricing_results ORDER BY date DESC LIMIT 100",
                    conn
                )
                conn.close()

                if df.empty:
                    return jsonify({'error': 'No performance data available'}), 404

                # Calculate metrics
                metrics = {
                    'total_calculations': len(df),
                    'average_quantum_advantage': df['quantum_advantage'].mean(),
                    'average_premium': df['premium'].mean(),
                    'average_expected_loss': df['expected_loss'].mean(),
                    'regions_covered': df['region'].nunique()
                }

                return jsonify(metrics)

            except Exception as e:
                return jsonify({'error': str(e)}), 500

        @self.app.route('/api/regions', methods=['GET'])
        def get_regions():
            """Get available regions"""
            return jsonify({
                'regions': self.quantum_system.config['regions'],
                'count': len(self.quantum_system.config['regions'])
            })

    def run(self, host='0.0.0.0', port=5000, debug=False):
        """
        Run the Flask application
        """
        logger.info(f"Starting Quantum Insurance API on {host}:{port}")
        self.app.run(host=host, port=port, debug=debug)

class DataQualityMonitor:
    """
    Monitor data quality and system performance
    """

    def __init__(self, db_path="climate_insurance.db"):
        self.db_path = db_path
        self.quality_thresholds = {
            'min_data_points': 20,
            'max_missing_percentage': 0.1,
            'rainfall_range': (0, 500),
            'temperature_range': (-20, 60),
            'ndvi_range': (-1, 1)
        }

    def check_data_quality(self, region=None):
        """
        Check data quality for regions
        """
        conn = sqlite3.connect(self.db_path)

        if region:
            df = pd.read_sql_query(
                "SELECT * FROM climate_data WHERE region = ?",
                conn, params=(region,)
            )
        else:
            df = pd.read_sql_query("SELECT * FROM climate_data", conn)

        conn.close()

        if df.empty:
            return {'status': 'error', 'message': 'No data available'}

        quality_report = {}

        # Check data completeness
        total_records = len(df)
        missing_data = df.isnull().sum().sum()
        missing_percentage = missing_data / (total_records * len(df.columns))

        quality_report['completeness'] = {
            'total_records': total_records,
            'missing_values': int(missing_data),
            'missing_percentage': missing_percentage,
            'status': 'good' if missing_percentage < self.quality_thresholds['max_missing_percentage'] else 'warning'
        }

        # Check data ranges
        for column, valid_range in [
            ('rainfall', self.quality_thresholds['rainfall_range']),
            ('temperature', self.quality_thresholds['temperature_range']),
            ('ndvi', self.quality_thresholds['ndvi_range'])
        ]:
            if column in df.columns:
                min_val, max_val = valid_range
                outliers = len(df[(df[column] < min_val) | (df[column] > max_val)])

                quality_report[f'{column}_range'] = {
                    'outliers': outliers,
                    'percentage': outliers / total_records,
                    'status': 'good' if outliers < total_records * 0.05 else 'warning'
                }

        # Overall quality score
        warning_count = sum(1 for metric in quality_report.values()
                          if metric.get('status') == 'warning')

        quality_report['overall'] = {
            'score': max(0, 1 - warning_count / len(quality_report)),
            'status': 'good' if warning_count == 0 else 'warning' if warning_count < 2 else 'poor'
        }

        return quality_report

    def generate_quality_report(self):
        """
        Generate comprehensive data quality report
        """
        report = {
            'timestamp': datetime.now().isoformat(),
            'regions': {}
        }

        # Get all regions
        conn = sqlite3.connect(self.db_path)
        regions = pd.read_sql_query("SELECT DISTINCT region FROM climate_data", conn)['region'].tolist()
        conn.close()

        for region in regions:
            report['regions'][region] = self.check_data_quality(region)

        # Overall system quality
        overall_scores = [data['overall']['score'] for data in report['regions'].values()]
        report['system_quality'] = {
            'average_score': np.mean(overall_scores),
            'regions_count': len(regions),
            'healthy_regions': sum(1 for score in overall_scores if score > 0.8)
        }

        return report

def demonstrate_data_pipeline():
    """
    Demonstrate the data pipeline components
    """
    print("Demonstrating Data Pipeline Components")
    print("=" * 50)

    # Sample region configurations
    regions = [
        {'name': 'Region_A', 'lat': 40.7128, 'lon': -74.0060},
        {'name': 'Region_B', 'lat': 34.0522, 'lon': -118.2437},
        {'name': 'Region_C', 'lat': 41.8781, 'lon': -87.6298}
    ]

    # 1. Data Ingestion
    print("\n1. Climate Data Ingestion")
    print("-" * 30)

    ingestion = ClimateDataIngestion()

    # Process sample data
    for region in regions:
        ingestion.process_and_store_data(region)

    print("Data ingestion completed")

    # 2. Data Quality Monitoring
    print("\n2. Data Quality Monitoring")
    print("-" * 30)

    monitor = DataQualityMonitor()
    quality_report = monitor.generate_quality_report()

    print(f"System quality score: {quality_report['system_quality']['average_score']:.3f}")
    print(f"Regions monitored: {quality_report['system_quality']['regions_count']}")
    print(f"Healthy regions: {quality_report['system_quality']['healthy_regions']}")

    # 3. API Demonstration
    print("\n3. API Components")
    print("-" * 30)

    # This would normally start the Flask app
    print("API endpoints configured:")
    print("- /api/health - Health check")
    print("- /api/pricing - Quantum pricing calculation")
    print("- /api/risk-assessment - Regional climate risk score")
    print("- /api/data/latest - Recent climate data")
    print("- /api/performance - System performance metrics")
    print("- /api/regions - Configured insurance regions")

    print("\nUse `QuantumInsuranceAPI(...).run()` to launch the API.")


In [ ]:
# Define a mock QuantumSystem with basic logic for now
class MockQuantumSystem:
    def __init__(self):
        self.config = {'regions': ['Region_A', 'Region_B', 'Region_C']}

    def process_region_pricing(self, region, date):
        return {
            'region': region,
            'date': date,
            'quantum_premium': round(np.random.uniform(20, 100), 2),
            'classical_premium': round(np.random.uniform(20, 100), 2),
            'quantum_expected_loss': round(np.random.uniform(10, 80), 2),
            'payout_probability': round(np.random.uniform(0.1, 0.9), 2),
            'quantum_advantage': round(np.random.uniform(-5, 15), 2)
        }

# Initialize and run the API
quantum_system = MockQuantumSystem()
api = QuantumInsuranceAPI(quantum_system)
api.run(debug=True)  # Launches the server
